In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *

import datetime as dt
import os

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_390/1118495462.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling P

# Example 4-day aggregation using speedmap data

* Export 4 days in April to support MTC Plan Bay Area 2050
* First, run speedmap pipeline for any days not already ran (can filter to subset of operators/districts if desired by using additional scripts and editing Makefile, see d4 example)

In [2]:
dates = [f'2023-04-{day}' for day in range(10, 14)]

In [3]:
dates

['2023-04-10', '2023-04-11', '2023-04-12', '2023-04-13']

In [4]:
progress_all_dates = []
for date in dates:
    df = pd.read_parquet(f'./_rt_progress_{date}.parquet')
    df = df >> filter(_.caltrans_district == '04 - Oakland')
    progress_all_dates += [df]
four_days = pd.concat(progress_all_dates)

Basic checks on organizations ran

In [5]:
# four_days >> count(_.organization_name, _.status, _.organization_itp_id)

In [6]:
# four_days >> filter(_.status == 'already_ran')

In [7]:
am_filter = {'period': 'am', 'filter_args': {'start_time': '06:00', 'end_time': '09:00'}}
mid_filter = {'period': 'mid', 'filter_args': {'start_time': '10:00', 'end_time': '14:00'}}
pm_filter = {'period': 'pm', 'filter_args': {'start_time': '15:00', 'end_time': '19:00'}}
all_filters = [am_filter, mid_filter, pm_filter]

In [8]:
from tqdm.notebook import tqdm

In [9]:
from rt_analysis import signal_tools

In [10]:
signal_tools.concatenate_speedmap_segments?

Signature:
signal_tools.concatenate_speedmap_segments(
    progress_df: pandas.core.frame.DataFrame = None,
    itp_id_list: list = None,
    analysis_date: datetime.datetime = None,
    pbar: tqdm.notebook.tqdm_notebook = None,
    filter_args: dict = None,
)
Docstring:
get polygon segments from legacy speedmap workflow, with relevant ids attached
relatively fast if already ran for date, slow otherwise

progress_df: see data_analyses/ca_transit_speed_maps
filter_dict: dict of args to RtFilterMapper.set_filter
File:      ~/data-analyses/rt_delay/rt_analysis/signal_tools.py
Type:      function

In [11]:
pbar = tqdm()

0it [00:00, ?it/s]

## loop over dates, using `signal_tools` to aggregate all operators in each time period

In [12]:
# for date in dates:
#     progress_df = pd.read_parquet(f'./_rt_progress_{date}.parquet')
#     progress_df = progress_df >> filter(_.caltrans_district == '04 - Oakland') >> filter(_.status == 'map_confirmed')
#     gdfs = {}
#     for time_filter in all_filters:
#         gdfs[time_filter['period']] = signal_tools.conacatenate_speedmap_segments(progress_df = progress_df, pbar=pbar, filter_args=time_filter['filter_args'])
#     for period in gdfs.keys():
#         gdfs[period].to_parquet(f'_{date}_{period}.parquet')

## concatenate/aggregate 4 day period

* start by aggregating metrics for each time of day period across all dates/operators
* then prefix am/mid/pm observations and concat

In [13]:
def read_all_dates(period):
    gdfs = []
    for date in dates:
        gdfs += [gpd.read_parquet(f'_{date}_{period}.parquet')]
    return pd.concat(gdfs)

In [14]:
all_am = read_all_dates('am')

In [16]:
all_am.columns

Index(['stop_id', 'stop_name', 'geometry', 'shape_id', 'stop_sequence',
       'route_id', 'route_short_name', 'direction_id', 'p50_mph', 'p20_mph',
       'p80_mph', 'fast_slow_ratio', 'trips_per_hour', 'miles_from_last',
       'time_formatted', 'gtfs_dataset_key', 'organization_name',
       'system_p50_median'],
      dtype='object')

In [17]:
all_pm = read_all_dates('pm')

In [18]:
all_mid = read_all_dates('mid')

In [19]:
group_cols = ['geometry', 'shape_id', 'stop_sequence',
       'route_id', 'route_short_name', 'direction_id', 'gtfs_dataset_key', 'organization_name',
             'miles_from_last']

In [20]:
prefix_cols = ['p50_mph', 'p20_mph',
       'p80_mph', 'fast_slow_ratio', 'trips_per_hour',
       'time_formatted', 'system_p50_median', 'n_weekdays']

In [21]:
def aggregate_prefix(period_concat_df, period):

    counts = (period_concat_df >> count(_.shape_id, _.stop_sequence, _.gtfs_dataset_key, _.organization_name)
          >> select(_.shape_id, _.stop_sequence, _.gtfs_dataset_key, _.organization_name, _.n_weekdays == _.n))
    all_period_agged = period_concat_df.groupby(group_cols).median().reset_index()
    all_period_agged = all_period_agged >> inner_join(_, counts, on = ['shape_id', 'stop_sequence', 'gtfs_dataset_key', 'organization_name'])
    prefix = period + '_'
    prefixed = [prefix + col for col in prefix_cols]
    rename_dict = dict(zip(prefix_cols, prefixed))
    all_period_agged = all_period_agged.rename(columns=rename_dict)
    
    return all_period_agged

In [22]:
am_prefixed = aggregate_prefix(all_am, 'am')

/tmp/ipykernel_390/3872286970.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_period_agged = period_concat_df.groupby(group_cols).median().reset_index()


In [23]:
pm_prefixed = aggregate_prefix(all_pm, 'pm')

/tmp/ipykernel_390/3872286970.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_period_agged = period_concat_df.groupby(group_cols).median().reset_index()


In [24]:
mid_prefixed = aggregate_prefix(all_mid, 'mid')

/tmp/ipykernel_390/3872286970.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_period_agged = period_concat_df.groupby(group_cols).median().reset_index()


In [25]:
joined = (am_prefixed >> inner_join(_, mid_prefixed, on=group_cols)
          >> inner_join(_, pm_prefixed, on=group_cols)
         
         )

In [26]:
joined.columns

Index(['geometry', 'shape_id', 'stop_sequence', 'route_id', 'route_short_name',
       'direction_id', 'gtfs_dataset_key', 'organization_name',
       'miles_from_last', 'am_p50_mph', 'am_p20_mph', 'am_p80_mph',
       'am_fast_slow_ratio', 'am_trips_per_hour', 'am_system_p50_median',
       'am_n_weekdays', 'mid_p50_mph', 'mid_p20_mph', 'mid_p80_mph',
       'mid_fast_slow_ratio', 'mid_trips_per_hour', 'mid_system_p50_median',
       'mid_n_weekdays', 'pm_p50_mph', 'pm_p20_mph', 'pm_p80_mph',
       'pm_fast_slow_ratio', 'pm_trips_per_hour', 'pm_system_p50_median',
       'pm_n_weekdays'],
      dtype='object')

## re-add stop id and stop name

* can't group on these since not present for virtual segments!

In [29]:
df = pd.concat([all_am, all_mid, all_pm]) >> distinct(_.shape_id, _.stop_sequence, _.gtfs_dataset_key,
                                                     _.stop_id, _.stop_name)

In [33]:
joined = joined >> left_join(_, df, on=['shape_id', 'stop_sequence', 'gtfs_dataset_key'])

In [34]:
joined >> count(_.stop_id, _.stop_name) >> arrange(-_.n)

,stop_id,stop_name,n
12011,NaN,NaN,1481
9566,7700100,Santa Rosa Transit Mall,14
9568,7700300,Mendocino Ave & Cherry St,14
9570,7700600,Mendocino Ave & Ridgeway,14
9567,7700200,B St & Ross St,12
...,...,...,...
12001,893222,Jefferson St at Pueblo Ave,1
12003,893224,American Canyon Senior Center (SB),1
12004,893228,Rio del Mar at Hwy 29 (7-11),1
12008,89337,Foster Rd at Indiana St North,1


In [35]:
joined = gpd.GeoDataFrame(joined)

In [36]:
joined.crs

<Projected CRS: EPSG:3310>
Name: NAD83 / California Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - California.
- bounds: (-124.45, 32.53, -114.12, 42.01)
Coordinate Operation:
- name: California Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [38]:
# (joined >> filter(_.stop_id.isna())).explore()

In [39]:
joined.to_parquet('d4_apr10_apr13_express_fixed.parquet')

In [40]:
pd.read_parquet('./d4_apr10_apr13_express_fixed.parquet').columns

Index(['geometry', 'shape_id', 'stop_sequence', 'route_id', 'route_short_name',
       'direction_id', 'gtfs_dataset_key', 'organization_name',
       'miles_from_last', 'am_p50_mph', 'am_p20_mph', 'am_p80_mph',
       'am_fast_slow_ratio', 'am_trips_per_hour', 'am_system_p50_median',
       'am_n_weekdays', 'mid_p50_mph', 'mid_p20_mph', 'mid_p80_mph',
       'mid_fast_slow_ratio', 'mid_trips_per_hour', 'mid_system_p50_median',
       'mid_n_weekdays', 'pm_p50_mph', 'pm_p20_mph', 'pm_p80_mph',
       'pm_fast_slow_ratio', 'pm_trips_per_hour', 'pm_system_p50_median',
       'pm_n_weekdays', 'stop_id', 'stop_name'],
      dtype='object')

In [41]:
joined.columns

Index(['geometry', 'shape_id', 'stop_sequence', 'route_id', 'route_short_name',
       'direction_id', 'gtfs_dataset_key', 'organization_name',
       'miles_from_last', 'am_p50_mph', 'am_p20_mph', 'am_p80_mph',
       'am_fast_slow_ratio', 'am_trips_per_hour', 'am_system_p50_median',
       'am_n_weekdays', 'mid_p50_mph', 'mid_p20_mph', 'mid_p80_mph',
       'mid_fast_slow_ratio', 'mid_trips_per_hour', 'mid_system_p50_median',
       'mid_n_weekdays', 'pm_p50_mph', 'pm_p20_mph', 'pm_p80_mph',
       'pm_fast_slow_ratio', 'pm_trips_per_hour', 'pm_system_p50_median',
       'pm_n_weekdays', 'stop_id', 'stop_name'],
      dtype='object')